In [11]:
import pandas as pd
import numpy as np
import json
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

### Read data 

In [6]:
with open("input_transp.json", mode="r", encoding='utf8') as file:
    input_data = json.load(file)
input_data

{'availabilities': {'S1': 14, 'S2': 26, 'S3': 11},
 'demands': {'C1': 5, 'C2': 13, 'C3': 15, 'C4': 17},
 'costs': [{'from': 'S1', 'to': 'C1', 'value': 10},
  {'from': 'S1', 'to': 'C2', 'value': 5},
  {'from': 'S1', 'to': 'C3', 'value': 20},
  {'from': 'S1', 'to': 'C4', 'value': 12},
  {'from': 'S2', 'to': 'C1', 'value': 12},
  {'from': 'S2', 'to': 'C2', 'value': 7},
  {'from': 'S2', 'to': 'C3', 'value': 12},
  {'from': 'S2', 'to': 'C4', 'value': 19},
  {'from': 'S3', 'to': 'C1', 'value': 6},
  {'from': 'S3', 'to': 'C2', 'value': 12},
  {'from': 'S3', 'to': 'C3', 'value': 16},
  {'from': 'S3', 'to': 'C4', 'value': 17}]}

### Set parameters

In [9]:
#create local variable to store input parameters
availabilities = input_data['availabilities']
demands = input_data['demands']
#dictionary of costs indexed by tuples (origin, destination)
costs = {(c['from'], c['to']): c['value'] for c in input_data['costs']}
costs

{('S1', 'C1'): 10,
 ('S1', 'C2'): 5,
 ('S1', 'C3'): 20,
 ('S1', 'C4'): 12,
 ('S2', 'C1'): 12,
 ('S2', 'C2'): 7,
 ('S2', 'C3'): 12,
 ('S2', 'C4'): 19,
 ('S3', 'C1'): 6,
 ('S3', 'C2'): 12,
 ('S3', 'C3'): 16,
 ('S3', 'C4'): 17}

In [18]:
availabilities.keys(), demands.keys()

(dict_keys(['S1', 'S2', 'S3']), dict_keys(['C1', 'C2', 'C3', 'C4']))

### Create a model

In [21]:
# Create a new model
model = gp.Model("Tranportation Model")

# Set sources and destinations
sources = list(availabilities.keys())
destinations = list(demands.keys())

# Set Parameters
availabilities = input_data['availabilities']
demands = input_data['demands']
#dictionary of costs indexed by tuples (origin, destination)
costs = {(c['from'], c['to']): c['value'] for c in input_data['costs']}

# Set decision variables:
X = model.addVars(sources, destinations,lb=0, name='Ship')

# Set objective function
model.setObjective(
    quicksum(costs[i, j]*X[i, j] for i in sources for j in destinations),
    GRB.MINIMIZE
)

# Add constraints

# availabilities constraints
for i in sources:
    model.addConstr(quicksum(X[i, j] for j in destinations) <= availabilities[i], name=f'Availabilities_{i}')
# demand constraints
for j in destinations:
    model.addConstr(quicksum(X[i, j] for i in sources) == demands[j], name=f'Demand_{j}')

# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 19.6.0 19H2026)

CPU model: Intel(R) Core(TM) i5-7360U CPU @ 2.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x78346a06
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+01]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.7900000e+02   1.600000e+01   0.000000e+00      0s
       2    5.2600000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.03 seconds (0.00 work units)
Optimal objective  5.260000000e+02


In [25]:

print(f"Objective_Value = {model.ObjVal:.2f}")
model.printAttr('X')

Objective_Value = 526.00

    Variable            X 
-------------------------
 Ship[S1,C2]            2 
 Ship[S1,C4]           12 
 Ship[S2,C2]           11 
 Ship[S2,C3]           15 
 Ship[S3,C1]            5 
 Ship[S3,C4]            5 
